In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge  # Régression Ridge (L2) ou Lasso si besoin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import mean_squared_error, r2_score

# Chargement des données
df = pd.read_csv("train_data.csv")  

# Calcul du prix moyen par zipcode
df["prix_moy_zipcode"] = df.groupby("zipcode")["prix"].transform("mean")

# Sélection des variables explicatives
X = df.drop(columns=["prix", "id", "date", "zipcode", "lat", "long"])  
y = df["prix"]

# Train/test split (20% pour le test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline avec régularisation Ridge
pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),  # Imputation des valeurs manquantes
    ("scaler", StandardScaler()),  # Standardisation
    ("feature_selection", SelectKBest(score_func=f_regression, k=5)),  # Sélection des meilleures variables
    ("model", Ridge())  # Régression Ridge (L2)
])

# Entraînement du modèle
pipeline.fit(X_train, y_train)

# Prédictions
y_pred = pipeline.predict(X_test)

# Évaluation du modèle
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"RMSE du modèle Ridge : {rmse:.2f}")
print(f"R² du modèle Ridge : {r2:.3f}")

# GridSearchCV pour optimiser la sélection des variables et la régularisation
param_grid = {
    "feature_selection__k": [3, 5, 7, 9, 11],  # Nombre de variables sélectionnées
    "model__alpha": [0.01, 0.1, 1, 10, 100]  # Force de la régularisation
}

# Recherche des meilleurs hyperparamètres avec validation croisée
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring="neg_root_mean_squared_error", n_jobs=-1)
grid_search.fit(X_train, y_train)

# Meilleurs paramètres et scores optimisés
best_rmse = -grid_search.best_score_  # Inversion car GridSearchCV retourne une valeur négative
best_r2 = grid_search.best_estimator_.score(X_test, y_test)  # Score R² du meilleur modèle

print(f"Meilleurs paramètres : {grid_search.best_params_}")
print(f"Meilleur RMSE en validation croisée : {best_rmse:.2f}")
print(f"R² du meilleur modèle : {best_r2:.3f}")


RMSE du modèle Ridge : 193955.87
R² du modèle Ridge : 0.727
Meilleurs paramètres : {'feature_selection__k': 11, 'model__alpha': 10}
Meilleur RMSE en validation croisée : 177996.15
R² du meilleur modèle : 0.772
